In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import os
from sklearn import preprocessing
import pandas as pd
import scipy.signal
import pickle
import csv
import random

import tensorflow as tf
import tensorflow.keras as tfk
from keras.models import Sequential

In [ ]:
# Random seed for reproducibility
seed = 27

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/AI_in_Medicine_Project/

Mounted at /content/drive
/content/drive/My Drive/AI_in_Medicine_Project


In [ ]:
# Load the test dataset
dataset = pd.read_csv('testset.csv', index_col=None)

In [ ]:
dataset = dataset.iloc[:, 1:]
dataset.head(5)

,lead_num,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,mean_RR,mean_Peaks,median_RR,median_Peaks,std_RR,std_Peaks,log_curr_RR,log_next_RR
0,0,-0.470550,-0.477702,-0.487722,-0.496410,-0.498187,-0.495045,-0.495088,-0.502621,-0.513491,-0.517090,-0.507614,-0.494721,-0.490354,-0.488069,-0.471642,-0.440591,-0.414297,-0.407646,-0.409313,-0.399141,-0.380798,-0.368158,-0.353794,-0.333373,-0.335073,-0.375830,-0.429196,-0.478005,-0.523547,-0.521129,-0.459054,-0.477967,-0.595404,-0.407452,0.337268,1.000000,0.743738,-0.202555,-0.895344,-1.0,-0.889771,-0.812241,-0.717591,-0.611672,-0.567719,-0.569040,-0.568071,-0.564729,-0.563428,-0.557747,-0.555172,-0.560556,-0.564691,-0.564679,-0.562487,-0.553999,-0.541560,-0.534668,-0.530675,-0.517481,-0.490932,-0.454381,-0.412746,-0.372402,-0.333278,-0.284305,-0.217441,-0.138851,-0.062715,-0.001533,210.056983,-0.008529,213.0,-0.05,35.465774,0.2235,4.465908,4.465908
1,0,-0.447087,-0.439602,-0.437994,-0.444811,-0.450578,-0.451561,-0.452348,-0.453012,-0.450939,-0.448383,-0.448260,-0.450995,-0.455320,-0.453937,-0.436891,-0.406693,-0.379862,-0.368414,-0.365576,-0.353838,-0.328711,-0.303531,-0.281483,-0.253137,-0.237164,-0.265776,-0.317539,-0.352402,-0.392376,-0.437884,-0.410412,-0.367230,-0.482989,-0.534615,-0.006788,0.852428,1.000000,0.146747,-0.760376,-1.0,-0.860068,-0.767798,-0.701803,-0.581652,-0.497202,-0.488073,-0.484343,-0.456736,-0.439885,-0.445156,-0.450912,-0.449797,-0.447824,-0.440996,-0.427573,-0.416906,-0.409775,-0.399524,-0.389360,-0.379555,-0.357627,-0.323682,-0.295054,-0.273123,-0.238114,-0.179437,-0.104372,-0.023281,0.056057,0.127976,210.056983,-0.008529,213.0,-0.05,35.465774,0.2235,4.465908,4.510860
2,0,-0.447987,-0.451945,-0.445739,-0.434309,-0.432391,-0.441265,-0.446860,-0.444473,-0.442836,-0.444031,-0.442791,-0.440173,-0.437736,-0.429100,-0.411182,-0.390598,-0.373642,-0.360688,-0.348646,-0.335839,-0.325530,-0.314559,-0.288231,-0.252751,-0.247810,-0.289049,-0.340677,-0.383314,-0.430268,-0.444627,-0.394864,-0.402427,-0.514989,-0.364780,0.336113,1.000000,0.758405,-0.204340,-0.912423,-1.0,-0.874081,-0.799260,-0.691722,-0.552547,-0.492592,-0.500466,-0.497947,-0.481950,-0.478902,-0.479837,-0.475223,-0.474203,-0.478268,-0.480467,-0.478374,-0.468713,-0.450341,-0.432987,-0.423516,-0.415557,-0.402598,-0.382111,-0.348609,-0.301248,-0.246183,-0.183144,-0.110471,-0.037302,0.030955,0.098358,210.056983,-0.008529,213.0,-0.05,35.465774,0.2235,4.510860,4.532599
3,0,-0.465087,-0.462209,-0.458494,-0.456639,-0.458695,-0.462747,-0.466194,-0.468414,-0.467042,-0.462972,-0.464274,-0.473299,-0.481104,-0.479420,-0.466376,-0.444907,-0.423313,-0.409123,-0.400369,-0.390103,-0.376505,-0.359678,-0.334412,-0.305229,-0.301474,-0.340782,-0.390282,-0.419383,-0.447264,-0.461788,-0.418507,-0.410561,-0.525708,-0.434866,0.239189,1.000000,0.882167,-0.076747,-0.868304,-1.0,-0.854975,-0.761754,-0.670456,-0.560430,-0.515164,-0.522396,-0.513074,-0.486095,-0.474269,-0.478975,-0.486120,-0.491082,-0.491279,-0.484368,-0.475595,-0.470762,-0.468588,-0.466802,-0.463464,-0.452221,-0.429985,-0.403445,-0.377180,-0.345623,-0.300697,-0.237088,-0.156870,-0.074929,-0.006465,0.046960,210.056983,-0.008529,213.0,-0.05,35.465774,0.2235,4.532599,4.615121
4,0,-0.475441,-0.472939,-0.464361,-0.459511,-0.462971,-0.470778,-0.477591,-0.479699,-0.477488,-0.475138,-0.475236,-0.478942,-0.485494,-0.485625,-0.468879,-0.440704,-0.418012,-0.408064,-0.401486,-0.386293,-0.364823,-0.348162,-0.332957,-0.308969,-0.292574,-0.309066,-0.346679,-0.385233,-0.430957,-0.455605,-0.417659,-0.413035,-0.523619,-0.436662,0.218545,1.000000,0.961538,0.045411,-0.799623,-1.0,-0.867460,-0.766765,-0.683503,-0.574191,-0.510458,-0.502701,-0.495543,-0.480204,-0.480536,-0.491832,-0.492965,-0.483495,-0.478973,-0.484002,-0.489645,-0.488951,-0.480488,-0.466589,-0.453536,-0.442950,-0.429697,-0.413191,-0.394074,-0.363200,-0.311694,-0.243074,-

In [ ]:
# Load the model
model = tfk.models.load_model('lstm_63_more_feat_smote_change_1')

In [ ]:
def calculate_signal_properties(signal, peaks):

    """ Given the peaks' locations and the signal values 
        compute 6 beat features: mean, medium and std for these two arrays 
    """

    rr_dif = []
    
    for i in range(len(peaks)-1):
        rr_dif_temp = peaks[i+1]-peaks[i]
        rr_dif.append(rr_dif_temp)
        
    # Mean
    mean_RR = np.mean(rr_dif)
    mean_Peaks = np.mean(signal) # Should be 0
    
    # Median
    median_RR = np.median(rr_dif)
    median_Peaks = np.median(signal)

    # Standard deviation
    std_RR = np.std(rr_dif)
    std_Peaks = np.std(signal)

    features = np.hstack([mean_RR, mean_Peaks, median_RR, median_Peaks, std_RR, std_Peaks])

    return features

In [ ]:
def calculate_beat_properties(i, peaks):

    """ Given the current peak location and current peak order 
        compute 2 beat features: current and next RR distance dilogarithm 
    """
    # Logarithm current RR distance
    if i==0:
        curr_RR = peaks[i+1]-peaks[i]
        next_RR= curr_RR
    elif i==len(peaks)-1:
        curr_RR = peaks[i] - peaks[i-1]
        next_RR = curr_RR
    else:
        curr_RR = peaks[i]-peaks[i-1]
        next_RR = peaks[i+1]-peaks[i]

    return np.hstack([np.log(curr_RR), np.log(next_RR)])

In [ ]:
from scipy.signal import resample

def downsample(signal_orig, rpeaks, signal_fs, fs_new=128):
    
    """ Given the signal and its annotation, downsample the signal and 
        modify the positions of the annotations 
    """
    fs = signal_fs
    
    new_num_samples = int(len(signal_orig)/fs*fs_new)
    downsamp_signal = np.array(resample(signal_orig, new_num_samples))
    
    downsamp_ann = rpeaks/(fs/fs_new)
    downsamp_ann = np.round(downsamp_ann).astype(int)

    return downsamp_signal, downsamp_ann

In [ ]:
def filter_signal(signal, fs):
    N = 4
    Range = [0.5, 35] 
    Wn = [r/(fs/2) for r in Range]
    b, a = scipy.signal.butter(N, Wn, 'bandpass')
    output_signal = scipy.signal.filtfilt(b, a, signal)
    return output_signal
    

In [ ]:
path = 'test_set_7/'
for _, _, filenames in os.walk(path):    
    for filename in filenames:
        
        # Take only .mat files
        if filename[-7:-4]=='rpk' or filename[-7:-4]=='ann':
            continue
        
        # Read three files for each signal 
        mat = scipy.io.loadmat(path+filename)
        mat_rpeaks = scipy.io.loadmat(path+filename[:-4] + '_rpk.mat')
        
        signal = mat['ecg']
        rpeaks = mat_rpeaks['rpeaks']
                    
        D = 35   # Distance around R peak 

        # ECG Segmentation and Beat Processing
        lead_pred = []
        for lead in [0, 1]:
            signal_lead = signal[:, lead]

            # Compute the current beat properties
            feature_temp = calculate_signal_properties(signal_lead, rpeaks)
            
             # Resample the signals to 128Hz
            fs = int(filename[-7:-4])
            if fs == 250:
                signal_lead, rpeaks = downsample(signal_lead, rpeaks, fs)

            # Filter the signals
            signal_lead = filter_signal(signal_lead, fs = 128)

            lead_inputs = []
            for i, position in enumerate(rpeaks):
                position = position[0]
                
                # Check the boundries
                if position-D < 0:
                    beat = signal_lead[:2*D]   
                elif position+D > len(signal_lead)-1:
                    beat = signal_lead[-2*D:]
                else:
                    beat = signal_lead[position-D:position+D] 

                # Normalize the beat between [-1,1]
                beat = (2 *(beat - min(beat))/(max(beat)-min(beat))) - 1

                # Compute beat properties
                beat_features = calculate_beat_properties(i, rpeaks)

                # Concat beat and its features
                input = np.hstack([lead, beat, feature_temp, beat_features])

                # Append all the inputs from one lead
                lead_inputs.append(input)
                df_lead_inputs = pd.DataFrame(lead_inputs)
 
            # Predict with this model
            predictions = model.predict(df_lead_inputs)

            # List that contains predictions from both leads
            lead_pred.append(predictions)

        # Compare the max prediction values and decide on the label
        labels_0 = np.argmax(lead_pred[0], axis=1)
        labels_1 = np.argmax(lead_pred[1], axis=1)
        maxs_0 = np.amax(lead_pred[0], axis=1)
        maxs_1 = np.amax(lead_pred[1], axis=1)

        signal_labels = []
        for i in range(len(labels_0)):
            # If the labels are the same take that label and if they 
            # are different take the one with the higher maximum value
            if labels_0[i]==labels_1[i]:
                signal_labels.append(labels_0[i])
            else:
                if maxs_0[i]>maxs_1[i]:
                    signal_labels.append(labels_0[i])
                else:
                    signal_labels.append(labels_1[i])

        # Convert list to a numpy array
        signal_labels = np.asarray(signal_labels)

        # Convert numberical labels into char labels N, V, S
        char_labels = np.array(len(labels_0)*[''])

        ind_0 = np.where(signal_labels==0)[0]
        ind_1 = np.where(signal_labels==1)[0]
        ind_2 = np.where(signal_labels==2)[0]

        char_labels[ind_0]='N'
        char_labels[ind_1]='S'
        char_labels[ind_2]='V'

        # Write ann.mat file 
        mymat={'num':char_labels}
        scipy.io.savemat('ann_files/' + filename[:-4] + '_ann.mat', 
                         mymat,
                         oned_as = 'column')
             